In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader
import numpy as np
import torchvision
from torchvision import datasets, models
import matplotlib.pyplot as plt
import time
import os
import copy
from utils.data import *
from transform import *
from dataset import PlanktonDataset
from model import PoseModel
from visualdl import LogWriter
from utils.vis import show_arrow
from utils.data import eval_euc_dists

%matplotlib inline

In [2]:
log_dir = "./log"
img_dir = '/data5/Plankton_wi18/rawcolor_db/images'
csv_filename = 'data/data_{}.csv'
phases = ['train', 'valid', 'test']
batch_size = 32
_GPU = 2

In [3]:
data_transform = {
    'train': transforms.Compose([
        Rescale((224, 224)),
        RandomHorizontalFlip(),
        RandomVerticalFlip(),
        ToTensor(),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        Rescale((224, 224)),
        ToTensor(),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        Rescale((224, 224)),
        ToTensor(),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

datasets = {x: PlanktonDataset(csv_file=csv_filename.format(x),
                               img_dir=img_dir,
                               transform=data_transform[x])
            for x in phases}

dataloaders = {x: DataLoader(datasets[x], batch_size=batch_size,
                             shuffle=True, num_workers=4)
               for x in phases}

dataset_sizes = {x: len(datasets[x]) for x in phases}

use_gpu = torch.cuda.is_available()

n_per_epoch = int(np.ceil(len(datasets['train']) / batch_size))

losses, errs = {}, {}

In [ ]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [4]:
def train_model(model, criterion, optimizer, scheduler, logger, num_epoch=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    
    for phase in ['train', 'valid']:
        try:
            with logger.mode(phase):
                if phase not in losses:
                    losses[phase] = logger.scalar('scalars/{}_loss'.format(phase))
                if phase not in errs:
                    errs[phase] = logger.scalar('scalars/{}_err'.format(phase))
            cnt = 0
        except:
            pass
    
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch, num_epoch))
        print('-' * 10)
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                model.train(False)
                
            running_loss = 0.0
            running_err = 0.0
            
            for data in dataloaders[phase]:
                inputs, target, coordinates = data['image'], data['target_map'], data['coordinates']
                
                if use_gpu:
                    inputs = Variable(inputs.cuda(_GPU))
                    target = Variable(target.cuda(_GPU))
                else:
                    inputs, target = Variable(inputs), Variable(target)
                    
                optimizer.zero_grad()
                
                outputs = model(inputs)
                loss = criterion(outputs, target)
                err = eval_euc_dists(outputs.cpu().data.numpy(), coordinates.numpy())
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    
                    losses[phase].add_record(cnt, loss.data[0])
                    errs[phase].add_record(cnt, err['average'])
                    cnt += 1
                    
                running_loss += loss.data[0] * inputs.size(0)
                running_err += err['average'] * inputs.size(0)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_err = running_err / dataset_sizes[phase]
            
            if phase == 'valid':
                losses[phase].add_record(epoch, epoch_loss)
                errs[phase].add_record(epoch, epoch_err)
            
            print('{} Loss: {:.4f}'. format(phase, epoch_loss))
            
            if phase == 'valid' and epoch_loss > best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
        
        save_checkpoint({
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer' : optimizer.state_dict(),
        }, is_best)
        
        print()
            
    time_elapsed = time.time() - since
    print('Training complete in {:0f}, {:0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))

    model.load_state_dict(best_model_wts)
    return model

In [5]:
model = PoseModel().cuda(_GPU)

criterion = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

logger = LogWriter(log_dir, sync_cycle=10)

In [6]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, logger, num_epoch=25)

Epoch 0/25
----------
train Loss: 0.0384
valid Loss: 0.0225

Epoch 1/25
----------
train Loss: 0.0213
valid Loss: 0.0207

Epoch 2/25
----------
train Loss: 0.0201
valid Loss: 0.0199

Epoch 3/25
----------
train Loss: 0.0193
valid Loss: 0.0192

Epoch 4/25
----------
train Loss: 0.0188
valid Loss: 0.0188

Epoch 5/25
----------
train Loss: 0.0182
valid Loss: 0.0183

Epoch 6/25
----------
train Loss: 0.0177
valid Loss: 0.0178

Epoch 7/25
----------
train Loss: 0.0174
valid Loss: 0.0177

Epoch 8/25
----------
train Loss: 0.0174
valid Loss: 0.0177

Epoch 9/25
----------
train Loss: 0.0173
valid Loss: 0.0176

Epoch 10/25
----------
train Loss: 0.0173
valid Loss: 0.0176

Epoch 11/25
----------
train Loss: 0.0172
valid Loss: 0.0176

Epoch 12/25
----------
train Loss: 0.0172
valid Loss: 0.0175

Epoch 13/25
----------
train Loss: 0.0171
valid Loss: 0.0175

Epoch 14/25
----------
train Loss: 0.0171
valid Loss: 0.0174

Epoch 15/25
----------
train Loss: 0.0171
valid Loss: 0.0174

Epoch 16/25
------